### k_fold.ipynb
- Alessandro Trincone
- Mario Gabriele Carofano

> ...

In [ ]:
import auxfunc
import constants

import numpy as np
from artificial_neural_network import NeuralNetwork
import dataset_function as df

import pprint
import time
from datetime import datetime
import copy

import os

...

In [ ]:
Xtrain, Ytrain, Xtest, Ytest = df.loadDataset(constants.COPPIE_TRAINING, constants.COPPIE_TEST)
Xtrain, Ytrain = df.split_dataset(Xtrain, Ytrain)

...

In [ ]:
print(f"\nK-fold cross-validation iniziato: {datetime.now().strftime(constants.DATE_TIME_FORMAT)}")
start_time = time.time()

for i in range(constants.DEFAULT_K_FOLD_VALUE):

    print(f"\nFold {i+1} di {constants.DEFAULT_K_FOLD_VALUE}")

    net = NeuralNetwork(
        784, 64, 10,
        hidden_act_funs=auxfunc.leaky_relu,
        output_act_fun=auxfunc.identity,
        e_fun=auxfunc.cross_entropy_softmax
    )

    training_fold = np.concatenate([fold for j, fold in enumerate(Xtrain) if j != i])
    training_labels = np.concatenate([fold for j, fold in enumerate(Ytrain) if j != i])
    validation_fold = Xtrain[i]
    validation_labels = Ytrain[i]

    if i == 0:
        best_fold_params = {
            "Fold"      : i+1,
            "Weights"   : copy.deepcopy(net.weights),
            "Biases"    : copy.deepcopy(net.biases),
            "Report"    : copy.deepcopy(net.training_report)
        }

    net.train(training_fold, training_labels, validation_fold, validation_labels)

    # scelta dei parametri per la rete che mostrano il minor errore di validazione
    if i == 0 or net.training_report.validation_error < best_fold_params["Report"].validation_error:
        best_fold_params = {
            "Fold"      : i+1,
            "Weights"   : copy.deepcopy(net.weights),
            "Biases"    : copy.deepcopy(net.biases),
            "Report"    : copy.deepcopy(net.training_report)
        }

    if constants.DEBUG_MODE:
        break

# end for i

end_time = time.time()
tot_time = end_time - start_time

print(f"\nK-fold cross-validation completato: {datetime.now().strftime(constants.DATE_TIME_FORMAT)}")
print(f"Tempo trascorso: {tot_time:.3f} secondi")

print(f"\nMiglior rete (fold): {best_fold_params['Fold']}")
print(repr(best_fold_params["Report"]))

...

In [ ]:
net.weights = copy.deepcopy(best_fold_params["Weights"])
net.biases = copy.deepcopy(best_fold_params["Biases"])
# net.save_network_to_file()

...

In [ ]:
# # TODO: invece di mostrare le predizioni sul terminale, mostrare un bar chart
# for test_example in zip(Xtest, Ytest):
#      label = np.argmax(test_example[1])
#      print(f"Ground truth: {constants.ETICHETTE_CLASSI[label]}")
#      net.predict(test_example[0])
#      df.show_image(test_example[0])